# In-Context Learning


In-context learning is a generalisation of few-shot learning where the LLM is provided a context as part of the prompt and asked to respond by utilising the information in the context.

* Example: *"Summarize this research article into one paragraph highlighting its strengths and weaknesses: [insert article text]”*
* Example: *"Extract all the quotes from this text and organize them in alphabetical order: [insert text]”*

A very popular technique that you will learn in week 5 called Retrieval-Augmented Generation (RAG) is a form of in-context learning, where:
* a search engine is used to retrieve some relevant information
* that information is then provided to the LLM as context


In this example we download some recent research papers from arXiv papers, extract the text from the PDF files and ask Gemini to summarize the articles as well as provide the main strengths and weaknesses of the papers. Finally we print the summaries to a local html file and as markdown.

In [2]:
!pip install requests bs4 google-generativeai pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 9.7 MB/s eta 0:00:00


In [3]:
import os
import requests
from bs4 import BeautifulSoup
import google.generativeai as genai
from urllib.request import urlopen, urlretrieve
from IPython.display import Markdown, display
from pypdf import PdfReader
from datetime import date
from tqdm import tqdm
from google.colab import userdata

In [4]:
# API_KEY = os.environ.get("GEMINI_API_KEY")
API_KEY = userdata.get('googleaistudio')
genai.configure(api_key=API_KEY)

We select those papers that have been featured in Hugging Face papers.

In [5]:
BASE_URL = "https://huggingface.co/papers"
page = requests.get(BASE_URL)
soup = BeautifulSoup(page.content, "html.parser")
h3s = soup.find_all("h3")

papers = []

for h3 in h3s:
    a = h3.find("a")
    title = a.text
    link = a["href"].replace('/papers', '')

    papers.append({"title": title, "url": f"https://arxiv.org/pdf{link}"})

Code to extract text from PDFs.

In [6]:
def extract_paper(url):
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)

    return text


def extract_pdf(url):
    pdf = urlretrieve(url, "pdf_file.pdf")
    reader = PdfReader("pdf_file.pdf")
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text


def printmd(string):
    display(Markdown(string))

In [7]:
LLM = "gemini-2.5-flash"
model = genai.GenerativeModel(LLM)

We use Gemini to summarize the papers.

In [10]:
for paper in tqdm(papers):
    try:
        paper["summary"] = model.generate_content("Create a formatted table with two columns: one column with the strengths of the article and one column with the weaknesses of the article. If you have missing data, leave the row empty." + extract_pdf(paper["url"])).text
    except:
        print("Generation failed")
        paper["summary"] = "Paper not available"

100%|██████████| 22/22 [13:18<00:00, 36.28s/it]


We print the results to a html file.

In [9]:
page = f"<html> <head> <h1>Daily Dose of AI Research</h1> <h4>{date.today()}</h4> <p><i>Summaries generated with: {LLM}</i>"
with open("papers.html", "w") as f:
    f.write(page)
for paper in papers:
    page = f'<h2><a href="{paper["url"]}">{paper["title"]}</a></h2> <p>{paper["summary"]}</p>'
    with open("papers.html", "a") as f:
        f.write(page)
end = "</head>  </html>"
with open("papers.html", "a") as f:
    f.write(end)

We can also print the results to this notebook as markdown.

In [11]:
for paper in papers:
    printmd("**[{}]({})**<br>{}<br><br>".format(paper["title"],
                                                paper["url"],
                                                paper["summary"]))

**[VCode: a Multimodal Coding Benchmark with SVG as Symbolic Visual
  Representation](https://arxiv.org/pdf/2511.02778)**<br>Here's a formatted table outlining the strengths and weaknesses of the article "VCode: a Multimodal Coding Benchmark with SVG as Symbolic Visual Representation":

| Strengths                                                     | Weaknesses (or challenges highlighted by the paper)                                                               |
| :------------------------------------------------------------ | :---------------------------------------------------------------------------------------------------------------- |
| Introduces VCode, a novel benchmark for visual-centric coding. | Visual-centric coding is underexplored, with prior work focusing on linguistic-centric or synthetic visual assets. |
| Advocates SVG as a compact, interpretable, and executable visual representation. | Conventional image representations (RGB pixels) provide limited symbolic abstraction for reasoning.           |
| Covers three challenging and diverse domains: general commonsense (MM-Vet), professional disciplines (MMMU), and visual-centric perception (CV-Bench). | Frontier VLMs struggle to generate faithful SVGs, revealing a persistent gap between language- and visual-centric coding. |
| Proposes CodeVQA, a novel evaluation protocol assessing symbolic fidelity by answering questions over rendered SVG. | The task is highly challenging; even the best SVG result remains well below the raw-image upper bound.          |
| Introduces VCoder, an agentic framework that significantly augments VLMs for visual coding. | Direct SVG generation from natural images is difficult due to: long-context code, visually-blind outputs, and weak visual fineness for low-level details. |
| VCoder integrates "Thinking with Revision" (iterative refinement) and "Acting with Visual Tools" (structured visual cues from detectors, segmenters, OCR). | Existing models perform poorly on professional knowledge (MMMU) and 3D reasoning (CV-Bench).                     |
| VCoder delivers a substantial +12.3-point overall gain over the top-performing Claude-4-Opus. | SVG specialist models (OmniSVG, StarVector) underperform significantly, indicating a gap between authored SVG corpora and natural image-derived SVGs. |
| Performance correlates positively with reasoning ability in VLMs. | Current coders are poorly adapted to direct visual input (Img2SVG), with performance improving when using an intermediate text caption. |
| Human studies show consistent patterns between humans and VLMs when reasoning over rendered SVGs, suggesting promise for human-like multimodal intelligence. | Abstraction to SVG inevitably leads to some information loss compared to original visual inputs.                   |
| The benchmark and code are publicly available.                 | Future work is needed to develop end-to-end vision-language coders with scalable training data for more faithful symbolic representations. |<br><br>

**[Don't Blind Your VLA: Aligning Visual Representations for OOD
  Generalization](https://arxiv.org/pdf/2510.25616)**<br>| Strengths                                                                                                                                                                                                                                                                                                                          | Weaknesses                                                                                                                                                                                                                                                                                                            |
| :-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| Provides a systematic study demonstrating representation degradation (collapse and attention sink) in Vision-Language-Action (VLA) models during naive fine-tuning.                                                                                                                                                                                  | The study primarily focuses on fine-tuning due to computational constraints, rather than full-scale VLA pretraining, which might limit generalizability to initial model development.                                                                                                                             |
| Introduces VL-Think, a novel diagnostic task suite designed to assess the transfer of Vision-Language (VL) knowledge and identify domain-specific forgetting in VLA models.                                                                                                                                                                          | The proposed alignment method only partially mitigates domain-specific forgetting in the VL-Think suite, with performance in several domains remaining largely unchanged.                                                                                                                                         |
| Proposes a simple, efficient, and lightweight Visual Representation Alignment method that mitigates degradation with negligible computational overhead.                                                                                                                                                                                               | The authors hypothesize that the effectiveness of the method may be limited by the modest size and diversity of the Supervised Fine-Tuning (SFT) dataset and the limited expressivity of LoRA updates, preventing broader gains across less frequent VL concepts.                                                     |
| Demonstrates significant and consistent improvements (up to a 10% relative gain) in Out-of-Distribution (OOD) generalization across semantic, vision, and execution axes.                                                                                                                                                                           | A trainable projector for alignment can lead to "representational bypassing," where the projector adapts to minimize loss instead of driving meaningful changes in the VLA's internal representations (though this is addressed by freezing the projector in the recommended setup).                                 |
| Conducts comprehensive quantitative (linear probing on ImageNet-100) and qualitative (attention maps, t-SNE visualization) analyses to support the findings and provide intuitive diagnostics of VL representation degradation.                                                                                                                       |                                                                                                                                                                                                                                                                                                                   |
| Features thorough ablation studies investigating the impact of different teacher models, alignment strategies, projector types, target layers, and loss functions, providing insights into optimal design choices.                                                                                                                                    |                                                                                                                                                                                                                                                                                                                   |
| Offers publicly available code, enhancing the reproducibility and accessibility of the research.                                                                                                                                                                                                                                                    |                                                                                                                                                                                                                                                                                                                   |
| Clarifies the trade-off between action fine-tuning and the degradation of VL representations, highlighting practical approaches to recover inherited VL capabilities and providing a practical recipe for building more robust VLAs.                                                                                                                 |                                                                                                                                                                                                                                                                                                                   |<br><br>

**[When Visualizing is the First Step to Reasoning: MIRA, a Benchmark for
  Visual Chain-of-Thought](https://arxiv.org/pdf/2511.02779)**<br>Here is a formatted table outlining the strengths and weaknesses of the article "When Visualizing is the First Step to Reasoning: MIRA, a Benchmark for Visual Chain-of-Thought":

| Strengths                                                                                                                                                                                                                                                                                                                                                                                            | Weaknesses                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| :--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Addresses a Critical Gap in MLLM Evaluation:** Introduces MIRA, a novel benchmark specifically designed to evaluate multimodal reasoning requiring *intermediate visual image generation*, a capability not adequately covered by existing benchmarks or text-only Chain-of-Thought (CoT) methods. This aligns with human "drawing to think."                                                              | **Reliance on Simulated Visual-CoT:** The Visual-CoT evaluation relies on *manually annotated* intermediate images because current MLLMs cannot yet generate these visuals themselves. This means the benchmark evaluates a model's *ability to utilize provided visuals*, not its capacity for autonomous "thinking while drawing." The paper itself acknowledges this as a "temporary solution."                                                                                                                                         |
| **Focus on Intrinsically Visual Reasoning:** MIRA's tasks are inherently challenging, involving complex structures, spatial relationships, and reasoning steps that are difficult to articulate or solve through language alone (e.g., tracking a die's movement on a board and summing face-down values).                                                                                                      | **Limited Efficacy of Text-CoT:** Text-only CoT methods offer minimal or even negative performance gains for strong models on MIRA, underscoring the inadequacy of purely linguistic reasoning for the benchmark's visually intensive tasks.                                                                                                                                                                                                                                                                                        |
| **High-Quality, Meticulously Curated Data:** Comprises 546 multimodal problems across 20 task types and four diverse domains (Euclidean Geometry, Physics-Based Reasoning, Abstract Spatial & Logical Puzzles, Causal Transformations), each annotated with gold-standard intermediate visual images and final answers, ensuring rigorous evaluation.                                                        | **Significant Performance Gap for Open-Weight Models:** Open-weight MLLMs show more limited improvements with Visual-CoT compared to closed-source models, suggesting an architectural or training-data-related gap in their ability to effectively leverage visual cues.                                                                                                                                                                                                                                                                    |
| **Novel Diagnostic Evaluation Protocol:** Implements a three-level evaluation framework (Direct Input, Text-CoT Reasoning, Simulated Visual-CoT Reasoning) to systematically measure model performance, isolate the contribution of visual cues, and gain insights into model failures.                                                                                                                       | **High Annotation Cost:** The manual creation and annotation of 546 multimodal problems with intermediate visual states is a labor-intensive process, which could be a barrier to scaling the benchmark or creating similar datasets.                                                                                                                                                                                                                                                                                                      |
| **Empirical Validation of Benchmark Challenge:** Demonstrates that even state-of-the-art closed-source MLLMs (e.g., GPT-5, Gemini 2.5 Pro) perform poorly on MIRA with direct inputs (none exceeding 20% accuracy), confirming its difficulty and the research gap it exposes.                                                                                                                                 | **Upper-Bound Probing Limitations:** Even with expanded search spaces (Pass@k) and specialized Text-CoT prompts, models show only modest improvements, indicating a "fundamental lack of capability" rather than mere reasoning errors, suggesting current MLLM architectures may lack the intrinsic visual reasoning needed to truly excel on MIRA.                                                                                                                                                                                          |
| **Strong Evidence for Visual-CoT's Efficacy:** Shows significant and consistent performance gains (average relative improvement of 33.7%) across models when intermediate visual cues are provided, highlighting the critical role of visual information in complex reasoning tasks.                                                                                                                               |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| **Reproducibility and Transparency:** Provides detailed experimental settings, prompt templates, and evaluation metrics in the appendix, ensuring the results can be replicated and built upon by the research community. The use of an LLM for copy-editing is also disclosed.                                                                                                                                |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |<br><br>

**[When Modalities Conflict: How Unimodal Reasoning Uncertainty Governs
  Preference Dynamics in MLLMs](https://arxiv.org/pdf/2511.02243)**<br>Here is a formatted table summarizing the strengths and weaknesses of the article:

| Strengths                                                                                             | Weaknesses                                                                                                                                                                                                         |
| :---------------------------------------------------------------------------------------------------- | :----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Novel Framework:** Proposes a new, principled framework decomposing modality following into "relative reasoning uncertainty" and "inherent modality preference."                                         | **Dataset Scope:** The core analysis relies on a "controllable toy dataset" with simplified tasks (e.g., color recognition of geometric shapes), which might limit direct generalizability to real-world, complex scenarios. |
| **Quantitative Measure:** Introduces the "balance point" to quantify a model's inherent modality preference, moving beyond coarse, macro-level statistics.                                            | **Preprint Status:** The article is a preprint, indicating it has not yet undergone formal peer review, which is a standard academic vetting process.                                                                   |
| **Empirical Discovery:** Uncovers a robust "universal monotonic law" demonstrating that the probability of following a modality decreases as its relative reasoning uncertainty increases.                 | **Model Scope:** While several MLLMs are evaluated (LLaVA, QwenVL families), the claim of a "universal law" could be further strengthened by testing an even broader range of MLLM architectures and scales.          |
| **Controllable Dataset:** Constructs a novel dataset that allows for systematic and independent manipulation of visual and textual reasoning difficulty.                                                     | **Uncertainty Metric Exploration:** While entropy is validated as a robust proxy for uncertainty, the article does not explore or compare it with other potential fine-grained uncertainty or confidence metrics.       |
| **Fine-grained Uncertainty Metric:** Validates output entropy as a reliable, model-centric measure of perceived unimodal reasoning uncertainty.                                                              |                                                                                                                                                                                                                    |
| **Mechanistic Insight:** Reveals and quantifies "internal oscillations" within model layers in ambiguous regions, providing a mechanistic explanation for external indecision.                                |                                                                                                                                                                                                                    |
| **Reconciling Prior Work:** Successfully explains and reconciles seemingly contradictory findings from previous macro-level analyses by disentangling capability from inherent preference.                  |                                                                                                                                                                                                                    |
| **Robustness and Generality:** Demonstrates the robustness and generality of the monotonic law across different data subsets and additional benchmarks (e.g., MC2, CLEVR-derived attribution dataset).      |                                                                                                                                                                                                                    |
| **Clarity of Concepts:** Clearly defines key concepts such as "modality following," "relative reasoning uncertainty," "inherent modality preference," and "oscillation," enhancing readability and understanding. |                                                                                                                                                                                                                    |
| **Strong Methodology:** The experimental setup is systematic, and the validation of the uncertainty metric is well-supported empirically.                                                                    |                                                                                                                                                                                                                    |<br><br>

**[Brain-IT: Image Reconstruction from fMRI via Brain-Interaction
  Transformer](https://arxiv.org/pdf/2510.25976)**<br>Here is a formatted table summarizing the strengths and weaknesses of the "BRAIN-IT: IMAGERECONSTRUCTION FROM FMRI VIABRAIN-INTERACTIONTRANSFORMER" article:

| Strengths                                                                                                                                                                                                                                                                                                                                                     | Weaknesses                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| :---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| Achieves image reconstructions from fMRI that faithfully reconstruct seen images, surpassing current State-of-the-Art (SotA) approaches both visually and by standard objective metrics.                                                                                                                                                                     | Reconstructions remain imperfect, with semantics and fine-grained details sometimes inaccurate.                                                                                                                                                                                                                                                                                                                                                     |
| Introduces "Brain-IT," a brain-inspired approach that addresses the challenge of lacking faithfulness to actual seen images in current methods.                                                                                                                                                                                                           | May reflect limitations of the fMRI signal itself, which could inherently limit reconstruction fidelity.                                                                                                                                                                                                                                                                                                                                             |
| Presents a novel "Brain Interaction Transformer" (BIT) that allows effective interactions between clusters of functionally-similar brain-voxels, which are shared by all subjects.                                                                                                                                                                         | The article mentions "failure cases" (Appendix Fig. S6) where either semantic content or visual layout is predicted inaccurately, leading to incorrect reconstructions. This implies specific scenarios where the model struggles.                                                                                                                                                                                                                           |
| BIT predicts two complementary localized patch-level image features: high-level semantic features (for semantic content) and low-level structural features (for coarse layout).                                                                                                                                                                         | Future work could explore more expressive feature spaces to address current method failures.                                                                                                                                                                                                                                                                                                                                                        |
| All model components are shared by all clusters & subjects, allowing efficient training with a limited amount of data.                                                                                                                                                                                                                                  | No explicit mention of computational cost or inference speed as a potential weakness, but complex models can sometimes have this issue (though the paper emphasizes "efficient training," it doesn't specify efficiency during reconstruction inference beyond training aspects). This is not explicitly stated as a weakness, but also not highlighted as a strength for inference. |
| Achieves results comparable to current methods trained on full 40-hour recordings with only 1-hour of fMRI data from a new subject, demonstrating efficient transfer-learning capabilities.                                                                                                                                                           |                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| First demonstration of high-quality reconstructions from as little as 15 minutes of fMRI data, highlighting strong generalization power.                                                                                                                                                                                                             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| Addresses limitations of prior methods, such as compressing all voxels into a single global fMRI embedding and relying on shared representations only at the scan level.                                                                                                                                                                                 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| Introduces a new approach to low-level image reconstruction from fMRI via Deep Image Prior (DIP), which accurately predicts the coarse image layout and forms a powerful input to the diffusion process.                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| The two-branch generation (semantic and low-level) provides complementary contributions, leading to reconstructions that are both semantically accurate and structurally faithful.                                                                                                                                                                    |                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| The BIT design, sharing network weights across all brain-voxels, facilitates effective cross-subject integration and adaptation to new subjects with limited data.                                                                                                                                                                                      |                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| Model is largely robust to the number of voxel clusters, maintaining strong performance across a wide range of cluster counts (provided the count is not too small).                                                                                                                                                                                 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| Functional clustering (used in Brain-IT) outperforms anatomical clustering, demonstrating the efficacy of its brain-inspired voxel grouping method.                                                                                                                                                                                                |                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| Incorporating "external images" (natural images without fMRI, with predicted fMRI responses) significantly enriches the training data and improves performance, especially beneficial for transfer learning.                                                                                                                                      |                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| The Brain-Transformer architecture, with Brain Tokenizer and Cross-Transformer, enables direct information flow from functional brain sub-regions to localized image features, potentially aiding neuroscientific applications.                                                                                                                    |                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |<br><br>

...

**[D2D: Detector-to-Differentiable Critic for Improved Numeracy in
  Text-to-Image Generation](https://arxiv.org/pdf/2510.19278)**<br>| Strengths                                                                                                                                                                                                                                                            | Weaknesses                                                                                                                                                                                                                                                                                                                                                                                     |
|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Addresses a significant limitation: Text-to-Image (T2I) models struggle with generating the correct number of objects.                                                                                                                                                               | High-density scenarios and generating very large counts (e.g., 11-20 objects) remain challenging, showing a significant drop in accuracy compared to low-density settings.                                                                                                                                                                                                                |
| Proposes a novel framework (D2D) to convert non-differentiable object detectors into differentiable critics.                                                                                                                                                                        | The method is not a layout-control approach, limiting its ability for fine-grained control over object placement or attribute binding.                                                                                                                                                                                                                                                          |
| Leverages the superior counting ability of detector-based models (especially in low-density scenarios) compared to existing regression-based methods, which are typically used as critics.                                                                                            | Certain object classes (e.g., wine glass, bottle – semi-transparent objects) show less significant improvement, suggesting potential limitations for visually complex or difficult object types.                                                                                                                                                                                                  |
| Designs custom high-curvature activation functions to transform non-differentiable detector logits into gradient-friendly soft binary indicators for count optimization.                                                                                                            | Achieving the absolute highest numeracy (e.g., with mixing weight `w=0`) can introduce "patchy visual artifacts"; a slightly lower performing `w=0.2` is chosen to balance numeracy with visual quality.                                                                                                                                                                                            |
| Introduces a backbone-agnostic Latent Modifier Network (LMN) that optimizes initial noise, enabling broader generalization across diverse T2I architectures (U-Net, DiT).                                                                                                          | An inference-time calibration phase for the Latent Modifier Network is crucial to avoid multi-color, patchy visual artifacts, indicating a need for careful initialization for each new input.                                                                                                                                                                                                  |
| Demonstrates consistent and substantial improvements in object counting accuracy across various generative backbones (SDXL-Turbo, SD-Turbo, Pixart-DMD).                                                                                                                             | Inherits biases of pre-trained diffusion base models and detectors, which is a general concern for such systems.                                                                                                                                                                                                                                                                                 |
| Evaluated across comprehensive benchmarks covering diverse numeracy generation scenarios, including single-object, multi-object, low-density, and high-density settings.                                                                                                            |                                                                                                                                                                                                                                                                                                                                                                                                |
| Effectively corrects both under-generations (42%) and over-generations (40%), significantly outperforming baselines (e.g., at least 16 percentage points over ReNO and TokenOpt correction rates).                                                                                    |                                                                                                                                                                                                                                                                                                                                                                                                |
| Achieves high numeracy with minimal degradation in overall image quality and minimal computational overhead (averaging 11-21 seconds compared to 28-100+ seconds for some baselines).                                                                                                  |                                                                                                                                                                                                                                                                                                                                                                                                |
| Numeracy improvements generalize across different detector backbones used as critics (OWLv2 and YOLOv9).                                                                                                                                                                            |                                                                                                                                                                                                                                                                                                                                                                                                |
| Improves numeracy across a wide variety of object categories (41 diverse objects tested on CoCoCount and D2D-Small).                                                                                                                                                                |                                                                                                                                                                                                                                                                                                                                                                                                |
| Can leverage open-vocabulary detectors (like OWLv2) for generating objects in the open set with minimal modifications.                                                                                                                                                               |                                                                                                                                                                                                                                                                                                                                                                                                |
| Effectively complements existing baselines, boosting numeracy when used in combination with methods like TokenOpt or ReNO.                                                                                                                                                             |                                                                                                                                                                                                                                                                                                                                                                                                |
| The Latent Modifier Network (LMN) specifically contributes to boosting the correction rate across various initial generation error levels (absolute error between requested and generated count).                                                                                   |                                                                                                                                                                                                                                                                                                                                                                                                |
| The proposed regularization term penalizes larger deviations from a Gaussian distribution more, allowing flexibility while maintaining stability during optimization, and is shown to be beneficial for numeracy.                                                                    |                                                                                                                                                                                                                                                                                                                                                                                                |<br><br>

**[LiveSecBench: A Dynamic and Culturally-Relevant AI Safety Benchmark for
  LLMs in Chinese Context](https://arxiv.org/pdf/2511.02366)**<br>Here's a formatted table outlining the strengths and weaknesses of the LiveSecBench article (and the benchmark it describes):

| Strengths of the Article / LiveSecBench | Weaknesses of the Article / LiveSecBench |
| :-------------------------------------- | :--------------------------------------- |
| Specifically designed for **Chinese-language LLM application scenarios**, addressing a critical gap in existing English-centric benchmarks. | The **LiveSecBench dataset is not publicly disclosed**, hindering reproducibility, independent verification, and community contribution to the test set itself. |
| **Dynamic and continuously updated** benchmark, designed to counter LLM overfitting and keep pace with evolving AI security threats and attack methods. | Uses a **passive evaluation mechanism** where developers submit models for testing, rather than allowing direct access to the benchmark for self-evaluation. |
| Evaluates across **six critical dimensions** (Legality, Ethics, Factuality, Privacy, Adversarial Robustness, Reasoning Safety) rooted in Chinese legal and social frameworks. | **Reasoning Safety is not included in the overall average score calculation** for model ranking. |
| **Cultural and Contextual Relevance** is a core principle of dataset construction, with prompts reviewed by native Chinese speakers to trigger safety mechanisms effectively in the Chinese context. | The **Adversarial Robustness and Reasoning Safety dimensions do not contain dedicated test questions**; they are evaluated using reused questions from other dimensions, potentially limiting their specific depth. |
| Employs an **ELO rating system** and **Swiss-system pairing strategy** for fair, competitive, and efficient model evaluation and ranking. | **Planned new dimensions** (Text-to-Image Generation Safety, Agentic Safety) are not yet implemented in the current version (v251030). |
| Includes **Reasoning Safety** as an evaluation dimension, examining internal Chain-of-Thought processes beyond just final outputs. | Some models have **missing data for Reasoning Safety** (indicated by "—" in Table 1), suggesting incomplete evaluation across all dimensions for all tested models. |
| The benchmark is designed to incorporate **new threat vectors** and has a clear **update schedule** for dimensions and test questions. | |
| Evaluated a **diverse set of 22 models**, including both open-source and commercial API-based LLMs, providing a comprehensive landscape. | |
| Provides **detailed evaluation reports** to participating developers, offering granular insights into model performance and vulnerabilities. | |
| Offers a **publicly accessible leaderboard** for transparency of model performance. | |<br><br>

**[Discriminately Treating Motion Components Evolves Joint Depth and
  Ego-Motion Learning](https://arxiv.org/pdf/2511.01502)**<br>Here's a formatted table with the strengths and weaknesses of the article:

| Strengths                                                                                                                                                                                                                                                                                            | Weaknesses                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| :---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Novel Approach:** Introduces discriminative treatment of motion components leveraging distinct geometric regularities in rigid flows, benefiting both depth and ego-motion estimation.                                                                                                            | **Performance in Extreme Low Illumination:** Struggles to produce reliable predictions under extremely low illumination due to degradation of photometric cues and diminished contextual features for dense correspondence generation.                                                                                                                                                                                                                                                                          |
| **State-of-the-Art (SoTA) Performance:** Achieves SoTA performance in both visual odometry (VO) and depth estimation on multiple public datasets and a newly collected real-world dataset, particularly under challenging conditions where existing approaches fail.                                  | **Generalization to Unseen Complex Real-World Sequences:** Fails to generalize effectively to other unseen real-world sequences, especially those with significantly greater environmental and motion complexity than conventional driving scenarios. This suggests current network architecture and training strategies are insufficient for capturing generalizable patterns in such highly diverse and complex environments.                                                                                      |
| **Comprehensive Analysis:** Conducts phenomenological and theoretical analyses of rigid flow distinctions and limitations of indiscriminately mixing flow types for supervisory signal generation.                                                                                                    |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| **Enhanced Robustness & Stability:** Introduces optical axis and imaging plane alignment processes, reformulating joint learning into coaxial and coplanar forms, mitigating mixed motion type effects and enhancing depth learning robustness and stability.                                           |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| **Efficient PoseNet Refinement:** Discriminative treatment enables alignments using network outputs, eliminating the need for auxiliary pose estimation algorithms and refining each component of PoseNet outputs. Achieves performance comparable to SoTA hybrid methods with lower overhead.      |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| **Generalizable Framework:** DiMoDE is a general framework compatible with various depth and pose estimation models, showing architectural flexibility and consistent performance gains across different network architectures (e.g., Lite-Mono, D-HRNet, ResNet-18, SCIPaD PoseNet).                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| **Improved Convergence:** Achieves significantly smoother network convergence and training robustness for both depth estimation and VO across diverse challenging conditions, attributed to the incorporated geometric constraints and constraint cycles.                                            |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| **Reduced Error Accumulation:** Significantly alleviates error accumulation over long-term sequences, outperforming previous SoTA methods with substantial ATE reductions on KITTI Odometry training sequences.                                                                                      |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| **Effective Adaptation:** Demonstrates more effective adaptation to previously unseen environments through finetuning, mitigating error propagation and improving convergence.                                                                                                                      |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| **Robust in Adverse Conditions:** Shows significantly more robust ego-motion estimation on the MIAS-Odom dataset, handling overexposure, low illumination, severe motion blur, frequent camera shake, and large-degree turns. Achieves superior depth estimation in challenging DDAD and nuScenes datasets. |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| **Addresses Specific Depth Estimation Biases:** Effectively avoids overly distant predictions near the principal point and maintains consistent optimization across the depth map by addressing perspective scaling effects. Produces fine-grained predictions in texture-rich regions and reliability in texture-less areas. |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| **Robust to Hyperparameter Variations:** Performance is relatively robust to moderate adjustments of the weighting coefficients for loss terms (λ1, λ2).                                                                                                                                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |<br><br>

**[Reg-DPO: SFT-Regularized Direct Preference Optimization with GT-Pair for
  Improving Video Generation](https://arxiv.org/pdf/2511.01450)**<br>Here is a formatted table summarizing the strengths and weaknesses of the article:

| Strengths                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | Weaknesses                                                                                                                                                                                                                                                                                                                                                                                                 |
| :--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Addresses Key Challenges:** Systematically tackles costly data construction, unstable training, and heavy memory consumption in video DPO.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | **Limited Empirical Comparison to Other SOTA DPO Variants for Video:** While claiming to outperform "existing approaches" and achieve "state-of-the-art results," the experiments primarily compare Reg-DPO against a *baseline*, *SFT*, and *vanilla DPO*. Direct empirical comparisons against other specialized video DPO methods mentioned in the related work (e.g., VideoDPO, DensedDPO, OnlineVPO) are not provided, making its precise standing within the broader field less clear. |
| **GT-Pair for Efficient Data Construction:** Automatically builds high-quality preference pairs by using real videos as positives and model-generated videos as negatives, eliminating the need for external, costly, and time-consuming human annotation. It generates pairs with higher quality and stronger discriminability. Empirically proven to significantly outperform manually annotated pairs and baselines.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | **Potential for GT-Pair to become less discriminative with highly advanced base models:** The GT-Pair strategy relies on real videos "inherently surpassing generated videos" to create a clear preference gap. As video generation models advance and become more realistic, this gap might diminish, potentially reducing the discriminative power and effectiveness of GT-Pair data for pushing performance further at the very high end. |
| **Reg-DPO for Algorithmic Stability and Fidelity:** Integrates SFT loss as a regularization term into the DPO objective, significantly enhancing training stability and generation fidelity. It mitigates DPO's intrinsic instability, preventing rapid convergence, pronounced distribution shift, and model collapse (e.g., grid-like blurring artifacts). The SFT term provides joint supervision over positive and negative output distributions.                                                                                                                                                                                                                                                                                                                                                                                                                                                   | **Computational Overhead for Model Offloading (minor trade-off):** Although beneficial for memory saving, the model offloading technique introduces "some I/O overhead (∼1 minute per batch)," which, while deemed acceptable, could impact overall training speed in specific scenarios.                                                                                                              |
| **Dynamic Weighting Scheme for SFT Regularization:** Employs a dynamic weighting scheme (r=DGR by default) to balance preference learning with distribution consistency, ensuring both effectiveness and generalizability across scenarios. Detailed analysis of SFT regularization's impact on training dynamics and video quality is provided.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | **Initial Effort in GT-Pair Data Construction:** While GT-Pair automates *annotation*, the preceding stages of "data acquisition → filtering → normalization → video generation" for both real and model-generated videos still require significant initial effort and adherence to specific criteria to ensure high-quality and consistent data inputs.                                            |
| **Superior Performance:** Consistently outperforms existing approaches (Base, SFT, DPO) on both I2V and T2V tasks across multiple datasets, delivering superior video generation quality validated by both GSB human evaluations and VBench automatic metrics. It achieves balanced improvements across dimensions, avoiding issues like "dynamic decay" seen in pure SFT.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | **Sensitivity to Beta Parameter:** The DGR analysis indicates that the DPO gradient ratio is sensitive to the `beta` parameter, which can lead to volatile gradient changes, instability, or premature convergence if not tuned carefully. While Reg-DPO introduces dynamic regularization, the initial choice of `beta` for the underlying DPO objective still requires careful consideration.        |
| **Comprehensive Memory Optimization Scheme:** Combines FSDP with multiple memory-saving techniques (Flash Attention, Context Parallelism, Pair Parallelism, prompt pre-encoding, model offloading, graph/memory reclamation) to achieve nearly 3x higher training capacity than FSDP alone. It supports single-card training of ultra-large video models (e.g., 720x1280 resolution, 200 frames on 8x H100) and scales efficiently. This provides a reusable, system-level optimization paradigm.                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                                                                                                                                                                                                                                                                                                                                 |
| **Unified Training Framework:** Offers a flexible framework that supports I2V/T2V tasks, SFT/DPO modes, LoRA and full-parameter fine-tuning, as well as multi-resolution training, enhancing its applicability.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |                                                                                                                                                                                                                                                                                                                                                                                                 |
| **Extensive and Systematic Experiments:** Provides thorough empirical validation of the GT-Pair, the SFT regularization, and the overall superior performance of Reg-DPO through diverse experiments on multiple datasets and tasks, including detailed analysis of training dynamics and parameter studies.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |                                                                                                                                                                                                                                                                                                                                                                                                 |
| **Qualitative Improvements:** Demonstrates significant improvements in generated video quality, including enhanced visual consistency, reduction of micro-motion videos, improved generation stability, enhanced physical plausibility, stronger instruction adherence, and improved temporal coherence.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |                                                                                                                                                                                                                                                                                                                                                                                                 |
| **Planned Code and Data Release:** The stated intention to release data and code fosters reproducibility and community engagement.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                 |<br><br>

**[RiddleBench: A New Generative Reasoning Benchmark for LLMs](https://arxiv.org/pdf/2510.24932)**<br>Here is a formatted table summarizing the strengths and weaknesses of the article "RiddleBench: A New Generative Reasoning Benchmark for LLMs":

| Strengths of the Article                                         | Weaknesses of the Article                                        |
| :--------------------------------------------------------------- | :--------------------------------------------------------------- |
| Introduces RiddleBench, a new benchmark for complex, multi-step, and generative reasoning in LLMs. | LLMs, even top-tier ones, struggle significantly with the benchmark (e.g., ~60% accuracy for best models). |
| Focuses on assessing multifaceted reasoning (logical deduction, spatial awareness, constraint satisfaction) often overlooked by other benchmarks. | LLMs universally struggle with complex spatial reasoning (Seating Arrangements puzzles). |
| Comprises a substantial dataset of 1,737 challenging puzzles.    | LLMs demonstrate a failure to maintain a holistic mental model, leading to inconsistent reasoning. |
| Provides a comprehensive evaluation of leading proprietary and open-weight LLMs. | The "model-as-judge" paradigm for cross-model correction is often computationally intractable (55% timeout). |
| Analysis goes beyond simple accuracy to assess the reliability and robustness of LLM reasoning. | LLMs are poor at identifying flawed logic in other models (44.1% accuracy), often validating incorrect reasoning. |
| Structured analysis reveals specific LLM phenomena: hallucination cascades, poor self-correction, and fragile reasoning. | LLMs exhibit "hallucination cascades," uncritically adopting flawed reasoning from other models. |
| RiddleBench serves as a diagnostic tool for critical issues in LLMs. | LLMs show strong "error fixation," rarely reversing incorrect judgments (only 4.4% of cases). |
| Provides a valuable resource for guiding the development of more robust, reliable, and safer LLMs. | LLMs are poor at self-correction, failing to identify their own errors in 67.7% of trials. |
| Emphasizes compositional and integrated reasoning, better reflecting real-world logical puzzles. | LLMs exhibit a powerful "self-confirmation bias."             |
| High-fidelity dataset ensured by meticulous human verification and curation processes. | LLM reasoning is fragile and not robust to reordered constraints in prompts. |
| The benchmark is made publicly available under a permissive CC0 license, promoting broad research access. | LLM reasoning is fragile and not robust to the presence of irrelevant information (red herrings). |
| Highlights advanced reasoning as a key frontier for LLM development due to observed struggles. | Evaluation of proprietary models uses specific API versions (August 2025), so performance may change with updates. |
| Puzzles are uniquely suited for producing clear, diagnosable reasoning chains. | RiddleBench's curation from Indian exam materials may introduce cultural bias. |
| Identifies a unique and promising visual reasoning strategy in Gemini models (ASCII family trees). | High API costs and computation limited the scope of cross-model and self-correction experiments to a subset of models. |
| Aims to advance the study of reasoning in LLMs and guide the development of more dependable AI. | Potential for data contamination cannot be definitively ruled out for proprietary models with undisclosed training sets. |
|                                                                  | RiddleBench is currently limited to the English language.        |
|                                                                  | The benchmark size and scope could be expanded with more problem types or numbers. |<br><br>